In [3]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 12.1 MB/

In [5]:
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install -r requirements.txt

!python download_models.py

import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

import gradio as gr
from PIL import Image

# Define the function to process the image and generate the video
def process_image(img_path, model_name='males_model'):
    opt = TestOptions().parse(save=False)
    opt.display_id = 0  # do not launch visdom
    opt.nThreads = 1    # test code only supports nThreads = 1
    opt.batchSize = 1   # test code only supports batchSize = 1
    opt.serial_batches = True  # no shuffle
    opt.no_flip = True  # no flip
    opt.in_the_wild = True  # This triggers preprocessing of in the wild images in the dataloader
    opt.traverse = True  # This tells the model to traverse the latent space between anchor classes
    opt.interp_step = 0.05  # this controls the number of images to interpolate between anchor classes

    data_loader = CreateDataLoader(opt)
    dataset = data_loader.load_data()
    visualizer = Visualizer(opt)

    opt.name = model_name  # set model to 'males_model' or 'females_model'
    model = create_model(opt)
    model.eval()

    data = dataset.dataset.get_item_from_path(img_path)
    visuals = model.inference(data)

    os.makedirs('results', exist_ok=True)
    out_path = os.path.join('results', os.path.splitext(os.path.basename(img_path))[0].replace(' ', '_') + '.mp4')
    visualizer.make_video(visuals, out_path)

    return out_path




Cloning into 'Lifespan_Age_Transformation_Synthesis'...
remote: Enumerating objects: 618, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 618 (delta 4), reused 12 (delta 4), pack-reused 604
Receiving objects: 100% (618/618), 22.86 MiB | 40.64 MiB/s, done.
Resolving deltas: 100% (380/380), done.
/content/Lifespan_Age_Transformation_Synthesis/Lifespan_Age_Transformation_Synthesis/Lifespan_Age_Transformation_Synthesis
  0% 2.38k/213M [00:00<5:27:16, 10.9kB/s]Google Drive download failed.
Trying do download from alternate server

  0% 0.00/213M [00:00<?, ?B/s]
  0% 0.00/213M [00:00<?, ?B/s]
  0% 131k/213M [00:00<03:25, 1.04MB/s]
  5% 10.4M/213M [00:00<00:03, 54.6MB/s]
 10% 22.2M/213M [00:00<00:02, 81.6MB/s]
 16% 33.9M/213M [00:00<00:01, 95.1MB/s]
 21% 45.1M/213M [00:00<00:01, 101MB/s] 
 27% 57.1M/213M [00:00<00:01, 107MB/s]
 32% 68.9M/213M [00:00<00:01, 110MB/s]
 38% 80.7M/213M [00:00<00:01, 112MB/s]
 43% 92.5M/213M [00:00<

In [8]:
# Define the function to handle the Gradio inputs and outputs
def inference(img, model_choice):
    img_path = "input_image.png"
    img.save(img_path)
    video_path = process_image(img_path, model_name=model_choice)
    return video_path

# Create Gradio interface
iface = gr.Interface(
    fn=inference,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Radio(choices=['males_model', 'females_model'], label="Model Choice")
    ],
    outputs=gr.Video( label="Output Video"),
    title="Lifespan Age Transformation Synthesis",
    description="Upload an image and select a model to generate an age transformation video."
)

# Launch the Gradio interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5c98aeae5ef8fae07a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
